In [ ]:
pip install torch torchvision pillow transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [ ]:
import gradio as gr

def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c90b0b7eb0a44e2543.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="textbox", outputs="textbox")

demo.launch(share=True)  # Share your demo with just 1 extra parameter 🚀

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19611a8803545eb155.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
from transformers import (
    ViltProcessor, #ViLT 模型專用的 tokenizer + 影像前處理器
    ViltForQuestionAnswering, #ViLT 架構的 VQA 模型，用於「影像＋文字→答案分類」。
    ViltForMaskedLM #ViLT 架構的遮蔽語言模型頭，用於「填空（MLM）」任務。
)
#
import nltk
from nltk.corpus import wordnet
#wordnet 用來查同義詞。

# 1. 載入模型和 processor
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
vqa_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
mlm_model = ViltForMaskedLM.from_pretrained("dandelin/vilt-b32-mlm")
vqa_model.eval()
mlm_model.eval()

# 2. 下載 NLTK WordNet
nltk.download("wordnet")
nltk.download("omw-1.4")

# 3. 助手：生成獨立的 MLM prompt，遮蔽正確答案
def make_masked_prompt(answer: str):
    mask_tok = processor.tokenizer.mask_token #讀取 tokenizer 所用的遮蔽符號
    words = answer.split()
    masks = " ".join([mask_tok] * len(words))
    return f"In this image, the key appears {masks}."
  # ex.>>> make_masked_prompt("on the table")
  # answer.split() → ["on", "the", "table"]
  # len(words) → 3
  # masks → "[MASK] [MASK] [MASK]"
  # 最後回傳：
  # "In this image, the key appears [MASK] [MASK] [MASK]."

# 4. 主攻擊流程
def vqattack_full(image, question, correct_ans, M=10, eps=0.01):
    # 4.1 編碼乾淨對並取原始特徵
    clean_enc = processor(image, question, return_tensors="pt")
    with torch.no_grad():
        clean_out = vqa_model(**clean_enc, output_hidden_states=True)
    clean_feat = clean_out.hidden_states[-1].squeeze(0)

    # 4.2 初始化可微像素 & 問題
    pixel = clean_enc.pixel_values.clone().detach().requires_grad_(True)
    input_ids = clean_enc.input_ids.clone()

    # 4.3 label ↔ id 映射
    id2label = vqa_model.config.id2label
    label2id = {v: int(k) for k, v in id2label.items()}
    if correct_ans not in label2id:
        raise ValueError(f"答案 '{correct_ans}' 不在模型支持的答案列表中。")
    target_id = label2id[correct_ans]

    adv_q = question

    for m in range(1, M + 1):
        # a) 前向
        out = vqa_model(pixel_values=pixel, input_ids=input_ids,
                        output_hidden_states=True)
        probs = F.softmax(out.logits, dim=-1)
        p_corr = probs[0, target_id]
        pert_feat = out.hidden_states[-1].squeeze(0)

        # b) 潛在特徵攻擊
        clean_mean = clean_feat.mean(dim=0)
        pert_mean = pert_feat.mean(dim=0)
        loss_latent = F.cosine_similarity(clean_mean, pert_mean, dim=0)

        # c) Masked Answer Anti-Recovery
        mask_prompt = make_masked_prompt(correct_ans)
        mask_enc = processor(image, mask_prompt, return_tensors="pt")
        mlm_out = mlm_model(pixel_values=mask_enc.pixel_values,
                            input_ids=mask_enc.input_ids)
        mask_positions = (mask_enc.input_ids[0] == processor.tokenizer.mask_token_id).nonzero()
        if mask_positions.numel() == 0:
            raise RuntimeError("找不到任何 mask token，請檢查 make_masked_prompt")
        mask_pos = mask_positions[0].item()
        logits_mask = mlm_out.logits[0, mask_pos]
        token_ids = processor.tokenizer.convert_tokens_to_ids(correct_ans.split())
        correct_token_id = token_ids[0]
        p_recover = F.softmax(logits_mask, dim=-1)[correct_token_id]
        loss_anti = -torch.log(p_recover + 1e-8)

        # d) 合併 loss & backward
        total_loss = loss_latent + loss_anti
        vqa_model.zero_grad()
        mlm_model.zero_grad()
        total_loss.backward()

        # e) 更新影像 (FGSM)
        grad_sign = pixel.grad.sign()
        # 重要：不要對 pixel.grad.zero_()，直接 detach 並重設 grad
        pixel = (pixel + eps * grad_sign).clamp(0, 1).detach().requires_grad_(True)

        # f) 文字替換（每 3 步一次）
        if m % 3 == 0:
            for w in adv_q.split():
                syns = wordnet.synsets(w)
                if syns:
                    neww = syns[0].lemmas()[0].name()
                    adv_q = adv_q.replace(w, neww, 1)
                    break
            input_ids = processor(image, adv_q, return_tensors="pt").input_ids

        print(f"[Iter {m}] p_corr={p_corr:.3f}, loss_lat={loss_latent:.3f}, loss_anti={loss_anti:.3f}")
        print(" Q →", adv_q)

    return pixel.detach(), adv_q

# 5. Demo
if __name__ == "__main__":
    img = Image.open("key.jpg").convert("RGB")
    q = "Where is the key roughly located?"
    correct = "table"

    adv_img, adv_q = vqattack_full(img, q, correct, M=6, eps=0.02)
    from torchvision.transforms.functional import to_pil_image
    to_pil_image(adv_img[0]).save("adv_key.jpg")
    print("最終對抗問題：", adv_q)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


[Iter 1] p_corr=0.015, loss_lat=1.000, loss_anti=11.807
 Q → Where is the key roughly located?
[Iter 2] p_corr=0.005, loss_lat=0.898, loss_anti=11.807
 Q → Where is the key roughly located?
[Iter 3] p_corr=0.003, loss_lat=0.843, loss_anti=11.807
 Q → Where be the key roughly located?
[Iter 4] p_corr=0.000, loss_lat=0.797, loss_anti=11.807
 Q → Where be the key roughly located?
[Iter 5] p_corr=0.001, loss_lat=0.880, loss_anti=11.807
 Q → Where be the key roughly located?
[Iter 6] p_corr=0.002, loss_lat=0.899, loss_anti=11.807
 Q → Where beryllium the key roughly located?
最終對抗問題： Where beryllium the key roughly located?


In [ ]:
# Step 0: 安裝依賴（只需跑一次）
!pip install gradio torch torchvision transformers pillow

# Step 1: 載入必要套件並初始化模型
import gradio as gr
import torch
from PIL import Image
from transformers import ViltProcessor, ViltForQuestionAnswering

# 只做 inference，不需 download WordNet
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model     = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model.eval()

# Step 2: 定義推論函式
def vqa_infer(image: Image.Image, question: str):
    """
    Input:
      - image: PIL.Image
      - question: str
    Output:
      - answer: str
    """
    # Preprocess
    inputs = processor(image, question, return_tensors="pt")
    # Forward
    with torch.no_grad():
        outputs = model(**inputs)
    # 取最可能的那個答案類別
    answer_id = outputs.logits.argmax(-1).item()
    # 對應回文字
    answer = model.config.id2label[str(answer_id)]
    return answer

# Step 3: 定義 Gradio 介面
demo = gr.Interface(
    fn=vqa_infer,
    inputs=[
      gr.Image(type="pil", label="Upload Image"),
      gr.Textbox(lines=1, placeholder="Type your question here...", label="Question")
    ],
    outputs=gr.Textbox(label="Answer"),
    title="ViLT VQA Demo",
    description="Upload an image and ask a question; the model will answer."
)

# Step 4: 啟動服務 (share=True 會給你一個公開網址)
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b9499295456729373e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import torch
import torch.nn.functional as F
from PIL import Image
from transformers import (
    ViltProcessor,
    ViltForQuestionAnswering,
    ViltForMaskedLM
)
import nltk
from nltk.corpus import wordnet

# 確保 WordNet 可用
nltk.download("wordnet")
nltk.download("omw-1.4")

# 載入模型與 processor
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
vqa_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
mlm_model = ViltForMaskedLM.from_pretrained("dandelin/vilt-b32-mlm")
vqa_model.eval()
mlm_model.eval()

# 生成 MLM prompt 的 helper
def make_masked_prompt(answer: str):
    mask_tok = processor.tokenizer.mask_token
    masks = " ".join([mask_tok] * len(answer.split()))
    return f"In this image, the key appears {masks}."

# 完整的攻擊流程函式
def vqattack_full(image, question, correct_ans, M=6, eps=0.02):
    # 1. 取得乾淨樣本特徵
    clean_enc = processor(image, question, return_tensors="pt")
    with torch.no_grad():
        clean_out = vqa_model(**clean_enc, output_hidden_states=True)
    clean_feat = clean_out.hidden_states[-1].squeeze(0)

    # 2. 初始化可微 pixel & 問題
    pixel = clean_enc.pixel_values.clone().detach().requires_grad_(True)
    input_ids = clean_enc.input_ids.clone()

    # 3. label ↔ id
    id2label = vqa_model.config.id2label
    label2id = {v: int(k) for k,v in id2label.items()}
    if correct_ans not in label2id:
        raise ValueError(f"答案 '{correct_ans}' 不在模型支持清單中")
    target_id = label2id[correct_ans]

    adv_q = question
    for m in range(1, M+1):
        # forward
        out = vqa_model(pixel_values=pixel, input_ids=input_ids, output_hidden_states=True)
        probs = F.softmax(out.logits, dim=-1)
        p_corr = probs[0, target_id]
        pert_feat = out.hidden_states[-1].squeeze(0)

        # 潛在特徵攻擊 loss
        loss_latent = F.cosine_similarity(clean_feat.mean(0), pert_feat.mean(0), dim=0)

        # Masked Answer Anti-Recovery loss
        mask_prompt = make_masked_prompt(correct_ans)
        mask_enc = processor(image, mask_prompt, return_tensors="pt")
        mlm_out = mlm_model(pixel_values=mask_enc.pixel_values, input_ids=mask_enc.input_ids)
        mask_positions = (mask_enc.input_ids[0] == processor.tokenizer.mask_token_id).nonzero()
        if mask_positions.numel() == 0:
            raise RuntimeError("MLM prompt 中沒找到 [MASK] token")
        mask_pos = mask_positions[0].item()
        logits_mask = mlm_out.logits[0, mask_pos]
        token_ids = processor.tokenizer.convert_tokens_to_ids(correct_ans.split())
        p_recover = F.softmax(logits_mask, dim=-1)[token_ids[0]]
        loss_anti = -torch.log(p_recover + 1e-8)

        # 合併並反傳
        total_loss = loss_latent + loss_anti
        vqa_model.zero_grad(); mlm_model.zero_grad()
        total_loss.backward()

        # FGSM 更新像素
        grad_sign = pixel.grad.sign()
        pixel = (pixel + eps * grad_sign).clamp(0,1).detach().requires_grad_(True)

        # 同義詞替換
        if m % 3 == 0:
            for w in adv_q.split():
                syns = wordnet.synsets(w)
                if syns:
                    adv_q = adv_q.replace(w, syns[0].lemmas()[0].name(), 1)
                    break
            input_ids = processor(image, adv_q, return_tensors="pt").input_ids

    return pixel.detach(), adv_q

# Gradio 介面包裝
def gradio_attack(image, question, correct_answer):
    adv_tensor, adv_q = vqattack_full(image, question, correct_answer)
    adv_img = Image.fromarray((adv_tensor[0].permute(1,2,0).cpu().numpy()*255).astype('uint8'))
    return adv_img, adv_q

demo = gr.Interface(
    fn=gradio_attack,
    inputs=[
        gr.Image(type="pil", label="輸入影像"),
        gr.Textbox(lines=1, label="提問問題"),
        gr.Textbox(lines=1, label="正確答案")
    ],
    outputs=[
        gr.Image(type="pil", label="對抗影像"),
        gr.Textbox(label="對抗問題")
    ],
    title="VQAttack Demo",
    description="上傳影像+問題+正確答案，產生對抗範例"
)

demo.launch(share=True)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7a2a6e08bdd3c37a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


以下是簡化版本v1

In [ ]:
# 安装依赖（只需跑一次）
!pip uninstall -y torch torchvision torchaudio
!pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install --no-cache-dir transformers gradio pillow nltk

Found existing installation: torch 2.7.1+cpu
Uninstalling torch-2.7.1+cpu:
  Successfully uninstalled torch-2.7.1+cpu
Found existing installation: torchvision 0.22.1+cpu
Uninstalling torchvision-0.22.1+cpu:
  Successfully uninstalled torchvision-0.22.1+cpu
Found existing installation: torchaudio 2.7.1+cpu
Uninstalling torchaudio-2.7.1+cpu:
  Successfully uninstalled torchaudio-2.7.1+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 106.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1+cpu which is incompatible.


In [ ]:

# 下面开始代码
import gradio as gr
import torch
import torch.nn.functional as F
from PIL import Image
from transformers import (
    ViltProcessor,
    ViltForQuestionAnswering,
    ViltForMaskedLM
)
import nltk
from nltk.corpus import wordnet

# 确保 WordNet 可用
nltk.download("wordnet")
nltk.download("omw-1.4")

# 载入模型和 processor
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
vqa_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
mlm_model = ViltForMaskedLM.from_pretrained("dandelin/vilt-b32-mlm")
vqa_model.eval()
mlm_model.eval()

# Helper: 生成 MLM prompt
def make_masked_prompt(answer: str):
    mask_tok = processor.tokenizer.mask_token
    masks = " ".join([mask_tok] * len(answer.split()))
    return f"In this image, the key appears {masks}."

# VQAttack 主流程（和之前的一致）
def vqattack_full(image, question, correct_ans, M=6, eps=0.02):
    clean_enc = processor(image, question, return_tensors="pt")
    with torch.no_grad():
        clean_out = vqa_model(**clean_enc, output_hidden_states=True)
    clean_feat = clean_out.hidden_states[-1].squeeze(0)

    pixel = clean_enc.pixel_values.clone().detach().requires_grad_(True)
    input_ids = clean_enc.input_ids.clone()

    id2label = vqa_model.config.id2label
    label2id = {v: int(k) for k,v in id2label.items()}
    if correct_ans not in label2id:
        raise ValueError(f"答案 '{correct_ans}' 不受支持。")
    target_id = label2id[correct_ans]

    adv_q = question
    for m in range(1, M+1):
        out = vqa_model(pixel_values=pixel, input_ids=input_ids, output_hidden_states=True)
        probs = F.softmax(out.logits, dim=-1)
        pert_feat = out.hidden_states[-1].squeeze(0)

        # latent loss
        loss_latent = F.cosine_similarity(clean_feat.mean(0), pert_feat.mean(0), dim=0)

        # masked-answer loss
        mask_prompt = make_masked_prompt(correct_ans)
        mask_enc = processor(image, mask_prompt, return_tensors="pt")
        mlm_out = mlm_model(pixel_values=mask_enc.pixel_values, input_ids=mask_enc.input_ids)
        mask_positions = (mask_enc.input_ids[0] == processor.tokenizer.mask_token_id).nonzero()
        if mask_positions.numel() == 0:
            raise RuntimeError("MLM prompt 中找不到 [MASK]")
        mask_pos = mask_positions[0].item()
        logits_mask = mlm_out.logits[0, mask_pos]
        token_ids = processor.tokenizer.convert_tokens_to_ids(correct_ans.split())
        p_recover = F.softmax(logits_mask, dim=-1)[token_ids[0]]
        loss_anti = -torch.log(p_recover + 1e-8)

        total_loss = loss_latent + loss_anti
        vqa_model.zero_grad(); mlm_model.zero_grad()
        total_loss.backward()

        grad_sign = pixel.grad.sign()
        pixel = (pixel + eps * grad_sign).clamp(0,1).detach().requires_grad_(True)

        if m % 3 == 0:
            for w in adv_q.split():
                syns = wordnet.synsets(w)
                if syns:
                    adv_q = adv_q.replace(w, syns[0].lemmas()[0].name(), 1)
                    break
            input_ids = processor(image, adv_q, return_tensors="pt").input_ids

    return pixel.detach(), adv_q

# Gradio接口
def gradio_attack(image, question, correct_answer):
    # 1) 原始推論
    clean_inputs = processor(image, question, return_tensors="pt")
    with torch.no_grad():
        clean_logits = vqa_model(**clean_inputs).logits
    orig_id  = clean_logits.argmax(-1).item()
    orig_ans = vqa_model.config.id2label[orig_id]

    # 2) 生成对抗样本
    adv_tensor, adv_q = vqattack_full(image, question, correct_answer)
    adv_img = Image.fromarray((adv_tensor[0].permute(1,2,0).cpu().numpy()*255).astype('uint8'))

    # 3) 对抗样本推論
    adv_inputs = processor(adv_img, adv_q, return_tensors="pt")
    with torch.no_grad():
        adv_logits = vqa_model(**adv_inputs).logits
    adv_id  = adv_logits.argmax(-1).item()
    adv_ans = vqa_model.config.id2label[adv_id]

    # 4) 计算“accuracy”标志
    orig_correct = (orig_ans == correct_answer)
    adv_correct  = (adv_ans  == correct_answer)
    attack_success = orig_correct and (not adv_correct)

    # 5) 返回：对抗图、对抗问句，以及结果摘要
    summary = (
        f"Clean→ {orig_ans} ({'✔' if orig_correct else '✘'})\n"
        f"Adv → {adv_ans} ({'✔' if adv_correct else '✘'})\n"
        f"Attack Success: {attack_success}"
    )
    return adv_img, adv_q, summary

demo = gr.Interface(
    fn=gradio_attack,
    inputs=[
        gr.Image(type="pil", label="輸入影像"),
        gr.Textbox(lines=1, label="Question"),
        gr.Textbox(lines=1, label="正確答案")
    ],
    outputs=[
        gr.Image(type="pil", label="對抗影像"),
        gr.Textbox(label="對抗問題"),
        gr.Textbox(label="結果摘要")
    ],
    title="VQAttack with Accuracy",
    description="上傳影像/問題/正確答案，生成對抗樣本並顯示原始 vs 對抗 模型預測 & 攻擊是否成功"
)

demo.launch(share=True,debug=True)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fe9bfc28565779f621.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fe9bfc28565779f621.gradio.live


以下是正式版v1 補上Cross-modal Joint Attack、改為PGD更新、以及用 TextFooler (BERT‐Attack) 完整流程来做文字端扰动的示例实现

In [ ]:
pip install textattack


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ==============================================================================
# 步驟 0：安裝並更新所有必要的 Python 套件
# ==============================================================================
# 這個儲存格會一次性地安裝或升級專案所需的所有函式庫，
# 以確保版本相容性，並解決 Colab 環境中可能出現的依賴問題。

print("正在安裝/更新所有必要的套件，請稍候...")

# Explicitly uninstall conflicting packages or old versions if needed
# Add specific uninstall for transformers
!pip uninstall -y transformers flair # Uninstall flair as well, it will be reinstalled as a textattack dependency

!pip install --upgrade --no-cache-dir --quiet \
torch \
torchvision \
torchaudio \
transformers>=4.5.0 \
gradio \
pillow \
nltk \
textattack \
sentence-transformers \
accelerate

print("✅ 套件安裝/更新完成！")
print("\n🔥 重要提示：如果在 Google Colab 中運行，請務必點擊上方選單的「執行階段」->「重新啟動工作階段」")
print("           (Runtime -> Restart session)，然後再從下一個儲存格開始執行程式碼。")

正在安裝/更新所有必要的套件，請稍候...
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: flair 0.15.1
Uninstalling flair-0.15.1:
  Successfully uninstalled flair-0.15.1
✅ 套件安裝/更新完成！

🔥 重要提示：如果在 Google Colab 中運行，請務必點擊上方選單的「執行階段」->「重新啟動工作階段」
           (Runtime -> Restart session)，然後再從下一個儲存格開始執行程式碼。


v2版

In [ ]:
# ==============================================================================
# 步驟 0：安裝並更新所有必要的 Python 套件 (最終純淨版)
# ==============================================================================
# 這個儲存格會一次性地安裝或升級專案所需的所有函式庫。
# 我們使用單一指令，讓 pip 自動解決所有套件間的依賴關係，以達到最穩定的狀態。

print("正在安裝/更新所有必要的套件，這個過程可能需要幾分鐘，請稍候...")

!pip install --upgrade --no-cache-dir --quiet \
    torch \
    torchvision \
    torchaudio \
    transformers \
    gradio \
    pillow \
    nltk \
    textattack \
    sentence-transformers \
    accelerate

print("✅ 套件安裝/更新完成！")
print("\n🔥 重要提示：如果在 Google Colab 中運行，請務必點擊上方選單的「執行階段」->「重新啟動工作階段」")
print("           (Runtime -> Restart session)，然後再重新執行一次這個儲存格來運行下面的程式碼。")



In [ ]:
# ==============================================================================
# 專案主程式碼
# ==============================================================================
import gradio as gr
import torch
import torch.nn.functional as F
from PIL import Image
import nltk
import numpy as np
import os

# TextAttack 元件的詳細導入
from textattack import Attacker, AttackArgs
from textattack.datasets import Dataset
from textattack.models.wrappers import ModelWrapper
from textattack.goal_functions import UntargetedClassification
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR
from textattack.attack import Attack

from transformers import (
    ViltProcessor,
    ViltForQuestionAnswering,
    ViltForMaskedLM,
)

# —— 0. 下載相依資源，只需運行一次 ——
# 將 NLTK 資料下載到一個已知路徑，避免 Colab 環境問題
nltk_data_path = os.path.join(os.path.expanduser("~"), "nltk_data")
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)
nltk.data.path.append(nltk_data_path)

print("Downloading NLTK data...")
nltk.download('averaged_perceptron_tagger_eng')
nltk.download("wordnet", quiet=True, download_dir=nltk_data_path)
nltk.download("omw-1.4", quiet=True, download_dir=nltk_data_path)
nltk.download('averaged_perceptron_tagger', quiet=True, download_dir=nltk_data_path)
nltk.download('universal_tagset', quiet=True, download_dir=nltk_data_path)
# TextAttack 的 StopwordFilter 需要 'punkt'
nltk.download('punkt', quiet=True, download_dir=nltk_data_path)
print("NLTK download complete.")

# —— 1. 設定設備並載入 ViLT VQA 與 MLM 模型 ——
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

print("Loading ViLT models...")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
vqa_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device)
mlm_model = ViltForMaskedLM.from_pretrained("dandelin/vilt-b32-mlm").to(device)
vqa_model.eval()
mlm_model.eval()
print("Models loaded successfully.")

# —— 2. 自定義 TextAttack Wrapper (此部分已確認無誤) ——
class ViltTextAttackWrapper(ModelWrapper):
    def __init__(self, model, processor, image):
        super().__init__()
        self.model = model
        self.processor = processor
        self.image = image

    def __call__(self, text_list):
        if isinstance(text_list, str):
            text_list = [text_list]
        elif isinstance(text_list, tuple):
            text_list = list(text_list)

        batch_size = len(text_list)
        image_batch = [self.image] * batch_size

        enc = self.processor(
            images=image_batch, text=text_list, return_tensors="pt",
            padding=True, truncation=True,
        ).to(self.model.device)

        with torch.no_grad():
            logits = self.model(**enc).logits
            probabilities = F.softmax(logits, dim=-1)

        return probabilities.detach().cpu().numpy()

# —— 3. 助手：構造 MLM prompt (此部分已確認無誤) ——
def make_masked_prompt(answer: str, template="In this image, the key appears {masks}."):
    mask_tok = processor.tokenizer.mask_token
    masks = " ".join([mask_tok] * len(answer.split()))
    return template.format(masks=masks)

# —— 4. 攻擊流程重構為兩個獨立函式 ——

# 4a. 階段一：只進行 PGD 圖像攻擊
def pgd_image_attack(image, question, correct_ans, M=6, eps=0.02, pgd_steps=3, sigma_i=0.03):
    print("\n--- STAGE 1: Performing PGD Image Attack ---")
    clean_enc = processor(image, question, return_tensors="pt").to(device)
    with torch.no_grad():
        clean_out = vqa_model(**clean_enc, output_hidden_states=True)
    hs_clean = clean_out.hidden_states[-1].squeeze(0)
    txt_len = clean_enc.input_ids.size(1)
    img_len = hs_clean.size(0) - txt_len
    clean_img_feat = hs_clean[:img_len].mean(dim=0)
    clean_text_feat = hs_clean[img_len:].mean(dim=0)

    orig_pixel = clean_enc.pixel_values.clone().detach()
    pixel = orig_pixel.clone().detach().requires_grad_(True)
    input_ids = clean_enc.input_ids.clone()

    for m in range(1, M + 1):
        print(f"  > PGD Image Attack Step {m}/{M}")

        out = vqa_model(pixel_values=pixel, input_ids=input_ids, output_hidden_states=True)
        hs_pert = out.hidden_states[-1].squeeze(0)
        pert_img = hs_pert[:img_len].mean(dim=0)
        pert_text = hs_pert[img_len:].mean(dim=0)
        loss_lat = F.cosine_similarity(clean_img_feat, pert_img, dim=0)
        mask_prompt = make_masked_prompt(correct_ans)
        mask_text_enc = processor.tokenizer(text=mask_prompt, return_tensors="pt", add_special_tokens=True, padding=True, truncation=True).to(device)
        mlm_out = mlm_model(pixel_values=pixel, input_ids=mask_text_enc.input_ids, attention_mask=mask_text_enc.attention_mask)
        pos = (mask_text_enc.input_ids[0] == processor.tokenizer.mask_token_id).nonzero(as_tuple=False)[0].item()
        logits_m = mlm_out.logits[0, pos]
        tok_id_list = processor.tokenizer.convert_tokens_to_ids(correct_ans.split())
        tok_id = tok_id_list[0] if tok_id_list else processor.tokenizer.unk_token_id
        p_rec = F.softmax(logits_m, dim=-1)[tok_id]
        loss_ant = -torch.log(p_rec + 1e-8)
        loss_text = F.cosine_similarity(clean_text_feat, pert_text, dim=0)
        total_loss = loss_lat + loss_ant + loss_text

        if pixel.grad is not None:
            pixel.grad.data.zero_()
        total_loss.backward()
        if pixel.grad is not None:
            with torch.no_grad():
                for _ in range(pgd_steps):
                    grad_sign = pixel.grad.sign()
                    pixel.data = pixel.data - (eps / pgd_steps) * grad_sign
                    eta = torch.clamp(pixel.data - orig_pixel, -sigma_i, sigma_i)
                    pixel.data = torch.clamp(orig_pixel + eta, 0, 1)

        pixel = pixel.detach().clone().requires_grad_(True)

    print("--- STAGE 1: PGD Image Attack Finished ---")
    return pixel.detach()

# 4b. 階段二：在對抗圖像上，只進行一次 TextAttack
def text_attack_on_adv_image(adv_image_pil, question, correct_ans):
    print("\n--- STAGE 2: Performing TextAttack on Adversarial Image ---")
    label2id = {v: int(k) for k, v in vqa_model.config.id2label.items()}
    if correct_ans not in label2id:
        raise ValueError(f"答案 '{correct_ans}' 不在支援的答案列表中")
    target_id = label2id[correct_ans]

    wrapper = ViltTextAttackWrapper(vqa_model, processor, adv_image_pil)
    goal_function = UntargetedClassification(wrapper)
    constraints = [
        RepeatModification(), StopwordModification(),
        WordEmbeddingDistance(min_cos_sim=0.7),
        PartOfSpeech(allow_verb_noun_swap=True, tagset='universal'),
    ]
    transformation = WordSwapEmbedding(max_candidates=50)
    search_method = GreedyWordSwapWIR()
    manual_attack = Attack(goal_function, constraints, transformation, search_method)

    atk_args = AttackArgs(num_examples=1, disable_stdout=True, log_to_txt=None, log_to_csv=None)
    attacker = Attacker(manual_attack, Dataset([(question, target_id)]), atk_args)
    results_list = attacker.attack_dataset()

    adv_q = question
    if results_list and results_list[0].perturbed_result:
        new_adv_q = results_list[0].perturbed_result.attacked_text.text
        print(f"  > TextAttack found new question: '{new_adv_q}'")
        adv_q = new_adv_q
    else:
        print("  > TextAttack did not find a better question (or was skipped).")

    print("--- STAGE 2: TextAttack Finished ---")
    return adv_q


# —— 5. Gradio 接口 (更新為兩階段流程) ——
# —— 5. Gradio 接口 (增加梯度清理的最終版) ——
def gradio_attack(image, question, correct_answer):
    if not all([image, question, correct_answer]):
        return None, "", "請提供圖片、問題和正確答案。"

    # 1) 原始推理
    print("\n--- Performing clean inference ---")
    clean_in = processor(image, question, return_tensors="pt").to(device)
    with torch.no_grad():
        clean_logits = vqa_model(**clean_in).logits
    orig_id = clean_logits.argmax(-1).item()
    orig_ans = vqa_model.config.id2label[orig_id]

    # 2) 生成對抗樣本 (兩階段)
    # 階段一：圖像攻擊
    adv_tensor = pgd_image_attack(image, question, correct_answer)
    adv_img_pil = Image.fromarray(
        (adv_tensor[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    )

    # 【【【 最終的黑魔法修正 】】】
    # 在進入 STAGE 2 前，強制清除模型參數上可能殘留的梯度，以防萬一。
    print("\n--- Clearing model gradients before STAGE 2 ---")
    vqa_model.zero_grad()
    mlm_model.zero_grad()

    # 階段二：文字攻擊
    adv_q = text_attack_on_adv_image(adv_img_pil, question, correct_answer)

    # 3) 最終對抗樣本推理
    print("\n--- Performing final adversarial inference ---")
    adv_in = processor(adv_img_pil, adv_q, return_tensors="pt").to(device)
    with torch.no_grad():
        adv_logits = vqa_model(**adv_in).logits
    adv_id = adv_logits.argmax(-1).item()
    adv_ans = vqa_model.config.id2label[adv_id]

    # 4) 總結攻擊結果
    print("\n--- Attack summary ---")
    orig_corr = (orig_ans.lower() == correct_answer.lower())
    adv_corr = (adv_ans.lower() == correct_answer.lower())
    success = orig_corr and (not adv_corr)
    summary = (
        f"原始預測: {orig_ans} (是否正確: {'✔' if orig_corr else '✘'})\n"
        f"對抗預測: {adv_ans} (是否正確: {'✔' if adv_corr else '✘'})\n\n"
        f"攻擊成功: {success}"
    )
    print(summary)
    return adv_img_pil, adv_q, summary

demo = gr.Interface(
    fn=gradio_attack,
    inputs=[
        gr.Image(type="pil", label="輸入影像 (Input Image)"),
        gr.Textbox(lines=1, label="問題 (Question)"),
        gr.Textbox(lines=1, label="正確答案 (Correct Answer)")
    ],
    outputs=[
        gr.Image(type="pil", label="對抗影像 (Adversarial Image)"),
        gr.Textbox(label="對抗問題 (Adversarial Question)"),
        gr.Textbox(label="結果摘要 (Result Summary)")
    ],
    title="多模態對抗攻擊 VQAttack Demo (兩階段穩定版)",
    description="上傳一張圖片，並提供一個關於圖片的問題以及該問題的正確答案。模型將先對圖像進行攻擊，然後在被攻擊的圖像上再進行文字攻擊，以生成多模態對抗樣本。",
    # examples=[
    #     ["https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/vqa.png", "what is the boy doing?", "playing tennis"],
    # ],
    allow_flagging="never"
)

# 啟動 Gradio 服務
demo.launch(share=True, debug=True)

NLTK download complete.
Using device: cpu
Loading ViLT models...


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Models loaded successfully.


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9944562966c463d30b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


...

textattack: Unknown if model of class <class 'transformers.models.vilt.modeling_vilt.ViltForQuestionAnswering'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--- STAGE 1: PGD Image Attack Finished ---

--- Clearing model gradients before STAGE 2 ---

--- STAGE 2: Performing TextAttack on Adversarial Image ---
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



  0%|          | 0/1 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sy

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9944562966c463d30b.gradio.live


  0%|          | 0/1 [04:08<?, ?it/s]


v3 deep copy 確保pretrain model每次呼叫都是new model

In [ ]:
# ==============================================================================
# 步驟 0：安裝並更新所有必要的 Python 套件 (最終純淨版)
# ==============================================================================
# 這個儲存格會一次性地安裝或升級專案所需的所有函式庫。
# 我們使用單一指令，讓 pip 自動解決所有套件間的依賴關係，以達到最穩定的狀態。

print("正在安裝/更新所有必要的套件，這個過程可能需要幾分鐘，請稍候...")

!pip install --upgrade --no-cache-dir --quiet \
    torch \
    torchvision \
    torchaudio \
    transformers \
    gradio \
    pillow \
    nltk \
    textattack \
    sentence-transformers \
    accelerate

print("✅ 套件安裝/更新完成！")
print("\n🔥 重要提示：如果在 Google Colab 中運行，請務必點擊上方選單的「執行階段」->「重新啟動工作階段」")
print("           (Runtime -> Restart session)，然後再重新執行一次這個儲存格來運行下面的程式碼。")



正在安裝/更新所有必要的套件，這個過程可能需要幾分鐘，請稍候...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 153.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 199.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 116.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 152.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 126.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 156.5 MB/s eta 

In [ ]:
# ==============================================================================
# 專案主程式碼
# ==============================================================================
import gradio as gr
import torch
import torch.nn.functional as F
from PIL import Image
import nltk #做文字同義詞等處理
import numpy as np
import os
import copy # 【新導入】: 為了深拷貝，避免多次呼叫時互相污染

# TextAttack 元件的詳細導入 用來做文字端的同義詞替換攻擊
from textattack import Attacker, AttackArgs #負責執行攻擊流程
from textattack.datasets import Dataset #包裝 (輸入句子, 正解標籤)
from textattack.models.wrappers import ModelWrapper #將 ViLT 封裝成 TextAttack 可呼叫的介面
from textattack.goal_functions import UntargetedClassification #目標函式，不需要指定新的標籤，只要改變原本正解即可
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR
from textattack.attack import Attack

# HuggingFace Transformers：用 ViLT 做 VQA／MLM（遮蔽語言模型）
from transformers import (
    ViltProcessor,
    ViltForQuestionAnswering,
    ViltForMaskedLM,
)

# —— 0. 下載相依資源 ——
nltk_data_path = os.path.join(os.path.expanduser("~"), "nltk_data")
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)
nltk.data.path.append(nltk_data_path)

print("Downloading NLTK data...")
nltk.download("wordnet", quiet=True, download_dir=nltk_data_path)
nltk.download("omw-1.4", quiet=True, download_dir=nltk_data_path)
nltk.download('averaged_perceptron_tagger', quiet=True, download_dir=nltk_data_path)
nltk.download('universal_tagset', quiet=True, download_dir=nltk_data_path)
nltk.download('punkt', quiet=True, download_dir=nltk_data_path)
print("NLTK download complete.")

# —— 1. 設定設備並載入「原型」模型 ——
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

print("Loading ViLT models (prototypes)...")
# 這些是我們的「工廠原型」，永不被污染
global_processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa") #ViLT 專用的 tokenizer + image preprocessor
global_vqa_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device) #用於 VQA 推理的模型
global_mlm_model = ViltForMaskedLM.from_pretrained("dandelin/vilt-b32-mlm").to(device) #用於「遮蔽問句」計算 anti‐recovery 損失的 MLM 模型
global_vqa_model.eval()
global_mlm_model.eval()
print("Models loaded successfully.")

# —— 2. 自定義 TextAttack Wrapper ——
# 目的：將 ViLT VQA 包裝為 TextAttack 要求的「black‐box 百分比分布輸出」介面。

# 輸入：text_list（一批候選問句），固定對同一張 image 做 VQA 推理。

# 輸出：對每個候選問句的答案類別機率向量（NumPy array），供 TextAttack 計算哪句最能使正確答案機率下降。

class ViltTextAttackWrapper(ModelWrapper):
    def __init__(self, model, processor, image):
        self.model = model
        self.processor = processor
        self.image = image

    def __call__(self, text_list):
        if isinstance(text_list, str):
            text_list = [text_list]
        elif isinstance(text_list, tuple):
            text_list = list(text_list)

        batch_size = len(text_list)
        image_batch = [self.image] * batch_size

        enc = self.processor(
            images=image_batch, text=text_list, return_tensors="pt",
            padding=True, truncation=True,
        ).to(self.model.device)

        with torch.no_grad():
            logits = self.model(**enc).logits
            probabilities = F.softmax(logits, dim=-1)

        return probabilities.detach().cpu().numpy()

# —— 3. 助手函式 ——
# 根據 correct_ans 的詞數，產生同樣數量的 [MASK]，用在對抗損失的「Masked‐Answer Anti‐Recovery」。
def make_masked_prompt(processor, answer: str, template="In this image, the key appears {masks}."):
    mask_tok = processor.tokenizer.mask_token
    masks = " ".join([mask_tok] * len(answer.split()))
    return template.format(masks=masks)

# —— 4. 兩階段攻擊流程 (接收模型作為參數) ——
# 4.1 第一階段：對影像做 PGD 多步攻擊

def pgd_image_attack(vqa_model, mlm_model, processor, image, question, correct_ans, M=6, eps=0.02, pgd_steps=3, sigma_i=0.03):
    print("\n--- STAGE 1: Performing PGD Image Attack ---")
    clean_enc = processor(image, question, return_tensors="pt").to(device)
    with torch.no_grad():
        clean_out = vqa_model(**clean_enc, output_hidden_states=True)
    hs_clean = clean_out.hidden_states[-1].squeeze(0)
    txt_len = clean_enc.input_ids.size(1)
    img_len = hs_clean.size(0) - txt_len
    clean_img_feat = hs_clean[:img_len].mean(dim=0)
    clean_text_feat = hs_clean[img_len:].mean(dim=0)

    orig_pixel = clean_enc.pixel_values.clone().detach()
    pixel = orig_pixel.clone().detach().requires_grad_(True)
    input_ids = clean_enc.input_ids.clone()

    for m in range(1, M + 1):
        print(f"  > PGD Image Attack Step {m}/{M}")

        out = vqa_model(pixel_values=pixel, input_ids=input_ids, output_hidden_states=True)
        hs_pert = out.hidden_states[-1].squeeze(0)
        pert_img = hs_pert[:img_len].mean(dim=0)
        pert_text = hs_pert[img_len:].mean(dim=0)
        loss_lat = F.cosine_similarity(clean_img_feat, pert_img, dim=0)
        mask_prompt = make_masked_prompt(processor, correct_ans)
        mask_text_enc = processor.tokenizer(text=mask_prompt, return_tensors="pt", add_special_tokens=True, padding=True, truncation=True).to(device)
        mlm_out = mlm_model(pixel_values=pixel, input_ids=mask_text_enc.input_ids, attention_mask=mask_text_enc.attention_mask)
        pos = (mask_text_enc.input_ids[0] == processor.tokenizer.mask_token_id).nonzero(as_tuple=False)[0].item()
        logits_m = mlm_out.logits[0, pos]
        tok_id_list = processor.tokenizer.convert_tokens_to_ids(correct_ans.split())
        tok_id = tok_id_list[0] if tok_id_list else processor.tokenizer.unk_token_id
        p_rec = F.softmax(logits_m, dim=-1)[tok_id]
        loss_ant = -torch.log(p_rec + 1e-8)
        loss_text = F.cosine_similarity(clean_text_feat, pert_text, dim=0)
        total_loss = loss_lat + loss_ant + loss_text
        # Eq.3 (latent‐feature) + Eq.4 (masked‐answer) + Eq.5 (cross‐modal text) 三項損失合併，
        # 對像素做 PGD 多步更新，再 clip 回合法範圍。
        if pixel.grad is not None:
            pixel.grad.data.zero_()
        total_loss.backward()
        if pixel.grad is not None:
            with torch.no_grad():
                for _ in range(pgd_steps):
                    grad_sign = pixel.grad.sign()
                    pixel.data = pixel.data - (eps / pgd_steps) * grad_sign
                    eta = torch.clamp(pixel.data - orig_pixel, -sigma_i, sigma_i)
                    pixel.data = torch.clamp(orig_pixel + eta, 0, 1)

        pixel = pixel.detach().clone().requires_grad_(True)

    print("--- STAGE 1: PGD Image Attack Finished ---")
    return pixel.detach()

# 4.2 第二階段：對剛剛被擾動的影像，做文字同義詞攻擊
# 完整套用 TextFooler（又名 WordSwapEmbedding + GreedyWIR）同義詞替換步驟，
# 把原始問題改成一個能進一步降低正確答案機率的問句。

def text_attack_on_adv_image(vqa_model, processor, adv_image_pil, question, correct_ans):
    print("\n--- STAGE 2: Performing TextAttack on Adversarial Image ---")
    label2id = {v: k for k, v in vqa_model.config.id2label.items()}
    if correct_ans not in label2id:
        raise ValueError(f"答案 '{correct_ans}' 不在支援的答案列表中")
    target_id = label2id[correct_ans]

    wrapper = ViltTextAttackWrapper(vqa_model, processor, adv_image_pil)
    goal_function = UntargetedClassification(wrapper)
    constraints = [
        RepeatModification(), StopwordModification(),
        WordEmbeddingDistance(min_cos_sim=0.7),
        PartOfSpeech(allow_verb_noun_swap=True, tagset='universal'),
    ]
    transformation = WordSwapEmbedding(max_candidates=50)
    search_method = GreedyWordSwapWIR()
    manual_attack = Attack(goal_function, constraints, transformation, search_method)

    atk_args = AttackArgs(num_examples=1, disable_stdout=True, log_to_txt=None, log_to_csv=None)
    attacker = Attacker(manual_attack, Dataset([(question, target_id)]), atk_args)
    results_list = attacker.attack_dataset()

    adv_q = question
    if results_list and results_list[0].perturbed_result:
        new_adv_q = results_list[0].perturbed_result.attacked_text.text
        print(f"  > TextAttack found new question: '{new_adv_q}'")
        adv_q = new_adv_q
    else:
        print("  > TextAttack did not find a better question (or was skipped).")

    print("--- STAGE 2: TextAttack Finished ---")
    return adv_q

# —— 5. Gradio 主函式 ——
def gradio_attack(image, question, correct_answer):
    if not all([image, question, correct_answer]):
        return None, "", "請提供圖片、問題和正確答案。"

    # 【【【 最終核心修正：深拷貝模型 】】】
    # 為本次攻擊創建一個全新的、純淨的模型副本
    print("\nCreating a deep copy of models for this run to ensure stateless execution...")
    local_vqa_model = copy.deepcopy(global_vqa_model)
    local_mlm_model = copy.deepcopy(global_mlm_model)
    # processor 是無狀態的，不需要拷貝
    processor = global_processor

    # 1) 原始推理 (使用副本)
    print("\n--- Performing clean inference ---")
    with torch.no_grad():
        clean_in = processor(image, question, return_tensors="pt").to(device)
        clean_logits = local_vqa_model(**clean_in).logits
    orig_id = clean_logits.argmax(-1).item()
    orig_ans = local_vqa_model.config.id2label[orig_id]

    # 2) 生成對抗樣本 (兩階段，傳入副本)
    adv_tensor = pgd_image_attack(local_vqa_model, local_mlm_model, processor, image, question, correct_answer)
    adv_img_pil = Image.fromarray(
        (adv_tensor[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    )
    adv_q = text_attack_on_adv_image(local_vqa_model, processor, adv_img_pil, question, correct_answer)

    # 3) 最終對抗樣本推理 (使用副本)
    print("\n--- Performing final adversarial inference ---")
    with torch.no_grad():
        adv_in = processor(adv_img_pil, adv_q, return_tensors="pt").to(device)
        adv_logits = local_vqa_model(**adv_in).logits
    adv_id = adv_logits.argmax(-1).item()
    adv_ans = local_vqa_model.config.id2label[adv_id]

    # 4) 總結攻擊結果
    print("\n--- Attack summary ---")
    orig_corr = (orig_ans.lower() == correct_answer.lower())
    adv_corr = (adv_ans.lower() == correct_answer.lower())
    success = orig_corr and (not adv_corr)
    summary = (f"原始預測: {orig_ans} (是否正確: {'✔' if orig_corr else '✘'})\n"
               f"對抗預測: {adv_ans} (是否正確: {'✔' if adv_corr else '✘'})\n\n"
               f"攻擊成功: {success}")
    print(summary)
    return adv_img_pil, adv_q, summary

# —— 6. 啟動 Gradio 介面 ——
demo = gr.Interface(
    fn=gradio_attack,
    inputs=[
        gr.Image(type="pil", label="輸入影像 (Input Image)"),
        gr.Textbox(lines=1, label="問題 (Question)"),
        gr.Textbox(lines=1, label="正確答案 (Correct Answer)")
    ],
    outputs=[
        gr.Image(type="pil", label="對抗影像 (Adversarial Image)"),
        gr.Textbox(label="對抗問題 (Adversarial Question)"),
        gr.Textbox(label="結果摘要 (Result Summary)")
    ],
    title="多模態對抗攻擊 VQAttack Demo (兩階段穩定版)",
    description="上傳一張圖片，並提供一個關於圖片的問題以及該問題的正確答案。模型將先對圖像進行攻擊，然後在被攻擊的圖像上再進行文字攻擊，以生成多模態對抗樣本。",
    allow_flagging="never"
)

demo.launch(share=True, debug=True)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NLTK download complete.
Using device: cpu
Loading ViLT models (prototypes)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

...

textattack: Unknown if model of class <class 'transformers.models.vilt.modeling_vilt.ViltForQuestionAnswering'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.


--- STAGE 1: PGD Image Attack Finished ---

--- STAGE 2: Performing TextAttack on Adversarial Image ---


100%|██████████| 481M/481M [00:10<00:00, 46.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpl_wvmcn6.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



  0%|          | 0/1 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sy